In [113]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [114]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [115]:
# load dataset
dataframe = pandas.read_csv("DONNES/train_cap2018.csv")
dataset = dataframe.values
X = dataset[:,1:58].astype(float)
Y = dataset[:,59]

where_are_NaNs = numpy.isnan(X) #Remplace les valeurs NaN par des 0
X[where_are_NaNs] = 0

#print(X)
#print(Y)

In [116]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

#print(encoded_Y)
#list(encoder.inverse_transform(encoded_Y)) la démarche inverse

In [120]:
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(8, input_dim=57, activation='relu'))
	model.add(Dense(6, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [121]:
model = baseline_model()
model.fit(X, dummy_y, epochs=150, batch_size=24) #Attention aux paramètres, certains changement provoquent l'overfitting !

Epoch 1/150
27310/27310 [==============================] - 2s 60us/step - loss: 12.6712 - acc: 0.2116
Epoch 2/150
27310/27310 [==============================] - 1s 37us/step - loss: 12.0365 - acc: 0.2459
Epoch 3/150
27310/27310 [==============================] - 1s 37us/step - loss: 9.4337 - acc: 0.3624
Epoch 4/150
27310/27310 [==============================] - 1s 37us/step - loss: 4.6738 - acc: 0.4767
Epoch 5/150
27310/27310 [==============================] - 1s 37us/step - loss: 1.5022 - acc: 0.5300
Epoch 6/150
27310/27310 [==============================] - 1s 37us/step - loss: 1.1155 - acc: 0.5618
Epoch 7/150
27310/27310 [==============================] - 1s 37us/step - loss: 0.9272 - acc: 0.6355
Epoch 8/150
27310/27310 [==============================] - 1s 37us/step - loss: 0.8967 - acc: 0.6609
Epoch 9/150
27310/27310 [==============================] - 1s 37us/step - loss: 0.8780 - acc: 0.6680
Epoch 10/150
27310/27310 [==============================] - 1s 37us/step - loss: 0.8586 -

27310/27310 [==============================] - 1s 35us/step - loss: 0.7311 - acc: 0.7243
Epoch 82/150
27310/27310 [==============================] - 1s 36us/step - loss: 0.7288 - acc: 0.7233
Epoch 83/150
27310/27310 [==============================] - 1s 36us/step - loss: 0.7245 - acc: 0.7274
Epoch 84/150
27310/27310 [==============================] - 1s 36us/step - loss: 0.7279 - acc: 0.7226
Epoch 85/150
27310/27310 [==============================] - 1s 35us/step - loss: 0.7279 - acc: 0.7249
Epoch 86/150
27310/27310 [==============================] - 1s 39us/step - loss: 0.7282 - acc: 0.7257
Epoch 87/150
27310/27310 [==============================] - 1s 35us/step - loss: 0.7284 - acc: 0.7228
Epoch 88/150
27310/27310 [==============================] - 1s 33us/step - loss: 0.7282 - acc: 0.7255
Epoch 89/150
27310/27310 [==============================] - 1s 34us/step - loss: 0.7285 - acc: 0.7237
Epoch 90/150
27310/27310 [==============================] - 1s 34us/step - loss: 0.7251 - acc: 

In [122]:
# evaluate the model
scores = model.evaluate(X, dummy_y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

27310/27310 [==============================] - 1s 27us/step

acc: 73.79%


In [86]:
#predictions = model.predict(X)
# round predictions
#for x in predictions :
#    print(x)
#print(dummy_y)